In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import tqdm
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import export_graphviz
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings 
warnings.filterwarnings(action="ignore")
from sklearn.model_selection import GridSearchCV

# 데이터 불러오기

In [3]:
train = pd.read_csv('data/train.csv', index_col='ID')
test = pd.read_csv('data/test.csv', index_col='ID')

# 이상치, 결측치 제거

In [4]:
# 공백문자가 포함된 컬럼 warehouse_block 수정
train = train.rename(columns = {'Warehouse_block ': 'Warehouse_block'})
test = test.rename(columns = {'Warehouse_block ': 'Warehouse_block'})

In [5]:
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Shipzk', 'Ship')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Flightzk', 'Flight')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Roadzk', 'Road')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Ship', 'Ship')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Flight', 'Flight')
train['Mode_of_Shipment']=train['Mode_of_Shipment'].replace(' Road', 'Road')

In [6]:
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Shipzk', 'Ship')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Flightzk', 'Flight')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Roadzk', 'Road')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Ship', 'Ship')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Flight', 'Flight')
test['Mode_of_Shipment']=test['Mode_of_Shipment'].replace(' Road', 'Road')

In [7]:
train['Customer_rating']=train['Customer_rating'].replace(99, 3)

In [8]:
test['Customer_rating']=test['Customer_rating'].replace(99, 3)

In [9]:
train['Product_importance'] = train['Product_importance'].replace('mediumm', 'medium')
train['Product_importance'] = train['Product_importance'].replace('loww', 'low')
train['Product_importance'] = train['Product_importance'].replace('highh', 'high')

In [10]:
test['Product_importance'] = test['Product_importance'].replace('mediumm', 'medium')
test['Product_importance'] = test['Product_importance'].replace('loww', 'low')
test['Product_importance'] = test['Product_importance'].replace('highh', 'high')

In [11]:
# 그대로 사용

In [12]:
# 최빈값에 ? 대입
train['Product_importance'] = train['Product_importance'].replace('?','low')
test['Product_importance'] = test['Product_importance'].replace('?','low')

In [13]:
# customer_care_calls => 1576

In [14]:
train['Customer_care_calls']=train['Customer_care_calls'].fillna(0)
test['Customer_care_calls']=test['Customer_care_calls'].fillna(0)

In [15]:
train['Customer_care_calls']=train['Customer_care_calls'].astype('int64')
test['Customer_care_calls']=test['Customer_care_calls'].astype('int64')

In [16]:
# prior_purchases => 950 

In [17]:
train['Prior_purchases']=train['Prior_purchases'].fillna(0)
test['Prior_purchases']=test['Prior_purchases'].fillna(0)

In [18]:
#고유한 값으로 사용?

In [19]:
mask = train['Weight_in_gms'].isin(['?'])
del_wei_question=train[~mask]

In [20]:
del_wei_question['Weight_in_gms']=del_wei_question['Weight_in_gms'].astype('int64')

In [21]:
pivot = del_wei_question.pivot_table(values='Weight_in_gms',
                         index=['Customer_care_calls'],
                         aggfunc='mean'
                         )
pivot['Weight_in_gms']=pivot['Weight_in_gms'].astype('int64')

In [22]:
train['Weight_in_gms']=train['Weight_in_gms'].replace('?', '0')

In [23]:
test['Weight_in_gms']=test['Weight_in_gms'].replace('?', '0')

In [24]:
train['Weight_in_gms']=train['Weight_in_gms'].astype('int64')

In [25]:
test['Weight_in_gms']=test['Weight_in_gms'].astype('int64')

In [26]:
def fill_wei(data):
    if data['Weight_in_gms']==0:
        return pivot.loc[data['Customer_care_calls']]
    else:
        return data['Weight_in_gms']

In [27]:
train['Weight_in_gms'] = train.apply(fill_wei, axis=1).astype('int64')

In [28]:
test['Weight_in_gms'] = test.apply(fill_wei, axis=1).astype('int64')

In [29]:
# Discount_offered => 3531
# 0 컬럼삭제

In [30]:
train['Cost_of_the_Product']=train['Cost_of_the_Product'].replace(9999,216)

In [31]:
test['Cost_of_the_Product']=test['Cost_of_the_Product'].replace(9999,216)

In [32]:
# discount offered 널값 채워넣기

# 시각화

# 전처리

# train, test 데이터 분류

In [33]:
X_train = train.loc[:,:'Weight_in_gms']
y_train = train['Reached.on.Time_Y.N']
X_test = test

# 인코딩

In [34]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# 스케일링

In [35]:
from sklearn.preprocessing import StandardScaler

In [36]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [37]:
X_train_Scaled = scaler.transform(X_train)
X_test_Scaled = scaler.transform(X_test)

# 모델 선정 및 학습

In [38]:
!pip install lightgbm

In [39]:
!pip install xgboost

In [40]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [41]:
# cv파라미터에 사용할 kfold값 설정
kfold = KFold(n_splits=10, random_state=123, shuffle=True)

In [42]:
# lightgbm 모델
lgbm_model = LGBMClassifier(n_estimators=1000, learning_rate=0.01, random_state=123)
lgbm_model.fit(X_train_Scaled, y_train)

LGBMClassifier(learning_rate=0.01, n_estimators=1000, random_state=123)

In [43]:
# 정확도 교차검증
acc_result = cross_val_score(lgbm_model,
                            X_train_Scaled,
                            y_train,
                            cv=kfold
                            )
print(acc_result.mean())

0.6652380952380952


In [44]:
# f1스코어 교차검증
f1_result = cross_val_score(lgbm_model,
                           X_train_Scaled,
                           y_train,
                           cv=kfold,
                           scoring='f1'
                           )
print(f1_result.mean())

0.6885754191901409


In [45]:
# xgboost 모델
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.01, random_state=123)
xgb_model.fit(X_train_Scaled, y_train)

[16:26:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              random_state=123, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [46]:
# 정확도 교차검증
acc_result = cross_val_score(xgb_model,
                            X_train_Scaled,
                            y_train,
                            cv=kfold
                            )
print(acc_result.mean())

[16:26:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:26:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:26:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:26:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [47]:
# f1스코어 교차검증
f1_result = cross_val_score(xgb_model,
                           X_train_Scaled,
                           y_train,
                           cv=kfold,
                           scoring='f1'
                           )
print(f1_result.mean())

[16:26:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:26:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:26:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:26:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [48]:
# randomforest 모델

In [49]:
X_train_Scaled

array([[-0.58111355, -0.70655499, -1.5566352 , ..., -0.86184621,
        -0.997289  ,  0.997289  ],
       [ 0.94600049,  0.00263451,  0.30993669, ..., -0.86184621,
         1.00271837, -1.00271837],
       [-0.07207554, -1.41574448,  0.39289544, ..., -0.86184621,
        -0.997289  ,  0.997289  ],
       ...,
       [-1.59918958, -1.41574448,  1.86541326, ..., -0.86184621,
         1.00271837, -1.00271837],
       [-0.07207554, -1.41574448,  0.4551145 , ...,  1.16029982,
        -0.997289  ,  0.997289  ],
       [-0.07207554,  0.00263451,  1.07730513, ..., -0.86184621,
         1.00271837, -1.00271837]])

In [50]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=11)
rf_model.fit(X_train_Scaled, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
# 정확도 교차검증
acc_result = cross_val_score(rf_model,
                            X_train_Scaled,
                            y_train,
                            cv=kfold
                            )
print(acc_result.mean())

In [ ]:
# f1스코어 교차검증
f1_result = cross_val_score(rf_model,
                           X_train_Scaled,
                           y_train,
                           cv=kfold,
                           scoring='f1'
                           )
print(f1_result.mean())

In [ ]:
pre_lgbm=lgbm_model.predict(X_test_Scaled)

In [ ]:
pre_xgb=xgb_model.predict(X_test_Scaled)

In [ ]:
pre_rf=rf_model.predict(X_test_Scaled)

In [ ]:
result_submission = pd.read_csv('data/sampleSubmission.csv')

In [ ]:
result_submission['Reached.on.Time_Y.N'] = pre_xgb
result_submission.to_csv('submission_xgb.csv', index=False)

In [ ]:
result_submission['Reached.on.Time_Y.N'] = pre_lgbm
result_submission.to_csv('submission_lgbm.csv', index=False)

In [ ]:
result_submission['Reached.on.Time_Y.N'] = pre_rf
result_submission.to_csv('submission_rf.csv', index=False)